# Equilibrator for Metabolic Network Analysis: Thermodynamic Profiling and Enzyme-Cost Minimization

Moritz E. Beber & Elad Noor

COMBINE2020

## Fundamentals

The direction of net flux through a reaction $r$ is given by the sign of its transformed Gibbs free energy at physiological conditions $ \operatorname{sgn} \left( \Delta_r G' \right) $.

\begin{align}
\Delta_r G' & = \Delta_r G'^\circ + R T \cdot \ln \left( Q_r \right) \\
\ln \left( Q_r \right) & = \mathbf{ S }_r^{ \top } \mathbf{ x }
\end{align}

* $ \Delta_r G'^\circ $: standard transformed Gibbs energy of reaction $ r $ (assumes $ 1 ~ \mathrm{mol} \mspace{3mu} \mathrm{L}^{-1} $ concentrations)
* $ R $: gas constant
* $ T $: temperature
* $ Q_r $: reaction quotient
* $ \mathbf{ S }_r $: column vector of the stoichiometric matrix, representing reaction $ r $
* $ \mathbf{ x } $: vector of natural logarithm of metabolite concentrations

\begin{align}
\Delta_r G' & = \Delta_r G'^\circ + R T \cdot \mathbf{ S }_r^{ \top } \mathbf{ x } \\
\end{align}

* $ \Delta_r G' < 0 $: net flux proceeds in forward direction
* $ \Delta_r G' > 0 $: net flux proceeds in reverse direction
* $ \Delta_r G' = 0 $: reaction is in equilibrium

$ -\Delta_r G' $ is also called the _driving force_

## How do we obtain the standard transformed Gibbs energy of a reaction?

* Measure it $ \Delta_r G'^\circ = - R T \ln \left( K'_r \right) $ at apparent equilibrium
* Measure energy of compound formation $ \Delta_r G'^\circ \approx \mathbf{ S }_r^{ \top } \Delta_f G^\circ $ known as Reactant Contribution (RC) uses reference points
* Group Contribution (GC) $ \Delta_r G'^\circ \approx \mathbf{ S }_r^{ \top } \mathbf{G} \Delta_g G^\circ $
* Component Contribution (CC) combines RC and GC

1. Jankowski, Matthew D., Christopher S. Henry, Linda J. Broadbelt, and Vassily Hatzimanikatis. 2008. “Group Contribution Method for Thermodynamic Analysis of Complex Metabolic Networks.” Biophysical Journal 95 (3): 1487–99. https://doi.org/10.1529/biophysj.107.124784.
2. Noor, Elad, Arren Bar-Even, Avi Flamholz, Yaniv Lubling, Dan Davidi, and Ron Milo. 2012. “An Integrated Open Framework for Thermodynamics of Reactions That Combines Accuracy and Coverage.” Bioinformatics 28 (15): 2037–44. https://doi.org/10.1093/bioinformatics/bts317.
3. Noor, Elad, Hulda S. Haraldsdóttir, Ron Milo, and Ronan M. T. Fleming. 2013. “Consistent Estimation of Gibbs Energy Using Component Contributions.” PLOS Computational Biology 9 (7): e1003098. https://doi.org/10.1371/journal.pcbi.1003098.
4. Du, Bin, Zhen Zhang, Sharon Grubner, James T. Yurkovich, Bernhard O. Palsson, and Daniel C. Zielinski. 2018. “Temperature-Dependent Estimation of Gibbs Energies Using an Updated Group-Contribution Method.” Biophysical Journal 114 (11): 2691–2702. https://doi.org/10.1016/j.bpj.2018.04.030.


## eQuilibrator

<img src="../images/equilibrator_3_0.svg" style="height: 20cm; display: inline-block; align: center;"/>

## Database Content

<img src="../images/metanetx-4.0.png" style="height: 15cm; display: inline-block;" />

In [1]:
from equilibrator_api import ComponentContribution
from equilibrator_cache.models import Compound

In [2]:
cc = ComponentContribution()

Fragments already downloaded
Fragments already downloaded


In [3]:
print(f"{cc.ccache.session.query(Compound).count():,d}")

694,324


In [4]:
structures = [
    c.inchi_key[:-2]
    for c in cc.ccache.session.query(Compound.inchi_key).filter(Compound.inchi_key.isnot(None))
]
print(f"{len(set(structures)):,d}")

460,769


glucose = gluconate

## What can we use the transformed Gibbs energies for?

* Max-min driving force
* Enzyme cost minimization
* Reversibility index
* Many others

### Max-min driving force (MDF)

\begin{eqnarray}
    \text{maximize} & B \\
    \text{subject to} &  -\Delta_r \mathbf{G}' & \geq B \\
    & \Delta_r \mathbf{G}' &= \Delta_r \mathbf{G}'^\circ + RT \cdot \mathbf{S}^\top \cdot \mathbf{x} \\
    & \ln(\mathbf{c}_{\min}) &\leq \mathbf{x} \leq \ln(\mathbf{c}_{\max})
\end{eqnarray}

### Enzyme cost minimization (ECM)

\begin{align}
v(s, p, E) = E ~ \dfrac{k_{cat}^+ ~ s/K_s - k_{cat}^- ~ p/K_p}{1 + s/K_s + p/K_p}
\end{align}

\begin{align}
E(s, p) = v ~ \frac{1 + s/K_s + p/K_p}{k_{cat}^+ ~ s/K_s - k_{cat}^- ~ p/K_p}
\end{align}

\begin{align}
q(\mathbf{x}) = \sum_i h_{E_i} E_i(\mathbf{x})
\end{align}

### Reversibility Index

* Reversibility index $ \gamma $ represents the required change in concentration to reverse reaction direction
* $ \gamma = 1000 \equiv \ln ( \gamma ) \approx 7 $ represents our threshold of three orders of magnitude


Noor, Elad, Arren Bar-Even, Avi Flamholz, Yaniv Lubling, Dan Davidi, and Ron Milo. 2012. “An Integrated Open Framework for Thermodynamics of Reactions That Combines Accuracy and Coverage.” Bioinformatics 28 (15): 2037–44. https://doi.org/10.1093/bioinformatics/bts317.

In [5]:
from operator import attrgetter
from cobra.io import load_model
from equilibrator_cache import CompoundCache
from equilibrator_api.compatibility import map_cobra_reactions

In [6]:
model = load_model("e_coli_core")

In [7]:
reactions = sorted(set(model.reactions) - set(model.boundary), key=attrgetter("id"))
len(reactions)

75

In [8]:
mapping = map_cobra_reactions(cache=cc.ccache, reactions=reactions)

In [9]:
len(mapping)

75

In [10]:
str(mapping["PFK"])

'Compound(id=6, inchi_key=ZKHQWZAMYRWXGA-KQYNXXCUSA-J) + Compound(id=74386, inchi_key=BGWGXPAPYGQALX-ARQDHWQXSA-L) => Compound(id=4, inchi_key=GPRLSGONYQIRFK-UHFFFAOYSA-N) + Compound(id=10, inchi_key=XTWYTFMLZFPYCI-KQYNXXCUSA-K) + Compound(id=396, inchi_key=RNBGYGVWRKECFJ-VRPWFDPXSA-J)'

In [11]:
for rxn_id, eq_rxn in mapping.items():
    ln_gamma = cc.ln_reversibility_index(eq_rxn)
    rxn = model.reactions.get_by_id(rxn_id)
    if rxn.reversibility and abs(ln_gamma) > 7.0:
        print(rxn_id, rxn.reversibility, ln_gamma)
    elif not rxn.reversibility and abs(ln_gamma) < 7.0:
        print(rxn_id, rxn.reversibility, ln_gamma)

ACALDt True (13.9 +/- 1.0) dimensionless
ACt2r True (-206.3 +/- 0.8) dimensionless
AKGDH False (-3.8 +/- 0.5) dimensionless
AKGt2r True (-514.2 +/- 1.1) dimensionless
ATPS4r True (12.58 +/- 0.08) dimensionless
CO2t True (-325.4 +/- 2.4) dimensionless
D_LACt2 True (-250.0 +/- 1.8) dimensionless
ETOHt2r True (52.7 +/- 1.1) dimensionless
FBP False (-6.6 +/- 0.4) dimensionless
GLNS False (-2.06 +/- 0.11) dimensionless
GLUSy False (-6.43 +/- 0.13) dimensionless
GLUt2r True (-292.0 +/- 1.0) dimensionless
GND False (-1.1 +/- 0.5) dimensionless
H2Ot True inf
ICL False (-2.0 +/- 0.4) dimensionless
ME1 False (-0.8 +/- 0.5) dimensionless
ME2 False (-0.8 +/- 0.5) dimensionless
NADTRHD False (-0.00 +/- 0.07) dimensionless
NH4t True (60.1 +/- 0.9) dimensionless
PDH False (-4.6 +/- 0.4) dimensionless
PFK False (-4.47 +/- 0.31) dimensionless
PFL False (-4.00 +/- 0.30) dimensionless
PIt2r True (-865.8 +/- 0.6) dimensionless
PPCK False (-1.0 +/- 0.5) dimensionless
PPS False (-3.56 +/- 0.08) dimensionles

In [12]:
model.reactions.THD2

Reaction identifier,THD2
Name,NAD(P) transhydrogenase
Memory address,0x07f598d16ab20
Stoichiometry,2.0 h_e + nadh_c + nadp_c --> 2.0 h_c + nad_c + nadph_c 2.0 H+ + Nicotinamide adenine dinucleotide - reduced + Nicotinamide adenine dinucleotide phosphate --> 2.0 H+ + Nicotinamide adenine dinucleotide + Nicotinamide adenine dinucleotide phosphate -...
GPR,b1602 and b1603
Lower bound,0.0
Upper bound,1000.0


## Thank You & Discussion

Please reach out to us with questions, suggestions, or problems.

* Chat with us on <a href="https://gitter.im/equilibrator-devs/equilibrator-api">gitter.im</a>
* Open an issue on https://gitlab.com/equilibrator/equilibrator-api
* <a href="mailto:noor@imsb.biol.ethz.ch?subject=equilibrator"><img src="../images/envelope-solid.svg" alt="email" style="height: 1.2em; display: inline-block;" /> noor@imsb.biol.ethz.ch</a>
* <a href="mailto:moritz@unseenbio.com?subject=equilibrator"><img src="../images/envelope-solid.svg" alt="email" style="height: 1.2em; display: inline-block;" /> moritz@unseenbio.com</a>